# Hiking Trails, Columbia River Gorge 
### Cleaning And Merging Data

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# Files to Load 
trails_data_to_load = Path("resources/original_data/HikingTrails_TheGorge_extended.xlsx")
hazards_data_to_load = Path("resources/original_data/Trail_hazards_danger.csv")

# Read trails and hazards Data Files and store into Pandas DataFrames
trails_data = pd.read_excel(trails_data_to_load, engine='openpyxl', sheet_name='Sheet1')
hazards_data = pd.read_csv(hazards_data_to_load)


trails_data.head()

,Trail Name,Trail Type,Distance,High Point,Elevation Gain,Difficulty,Seasons,Family Friendly,Backpackable,Crowded
0,Ainsworth Loop Hike,Loop,0.5 miles,150 feet,85 feet,Easy,All year,Yes,No,No
1,Aldrich Butte Hike,Out and Back,13.8 miles round trip,1140 feet,2405 feet,Moderate,All Season,No,No,No
2,Aldrich Butte-Cedar Falls Loop Hike,Lollipop loop,16.4 miles round trip,"1,140 feet",3105 feet,Difficult,Year round,No,No,No
3,Angels Rest Hike,Out and Back,4.8 miles round trip,1640 feet,1475 feet,Moderate,All Season,Yes,No,Yes
4,Angels Rest-Devils Rest Loop Hike,Loop,10.8 miles,2435 feet,3040 feet,Moderate,All Season,Yes,No,Yes


In [2]:
# checking for columns names for future references and editing -->trails df
print(trails_data.columns)

Index(['Trail Name', 'Trail Type', 'Distance', 'High Point', 'Elevation Gain',
       'Difficulty', 'Seasons', 'Family Friendly', 'Backpackable', 'Crowded'],
      dtype='object')


In [3]:
# checking for columns names for future references and editing --> hazards df
print(hazards_data.columns)

Index(['Name', 'Rattlesnakes', 'Ticks', 'Posionivy', 'Falling'], dtype='object')


## Data inspection

In [4]:
# Calculating the total number of unique trails
#"nunique"- number of unique values
trails_count = trails_data['Trail Name'].nunique()
print(f"Total number of unique trails: {trails_count}")

Total number of unique trails: 172


In [5]:
# Columns inspection for data types and check for missing values
print(trails_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Trail Name       172 non-null    object
 1   Trail Type       171 non-null    object
 2   Distance         172 non-null    object
 3   High Point       168 non-null    object
 4   Elevation Gain   171 non-null    object
 5   Difficulty       172 non-null    object
 6   Seasons          172 non-null    object
 7   Family Friendly  172 non-null    object
 8   Backpackable     171 non-null    object
 9   Crowded          171 non-null    object
dtypes: object(10)
memory usage: 13.6+ KB
None


As you can see, there’s some missing data scattered throughout, and the data format isn’t quite what we need. We’ll come back to filling in the missing data later, after cleaning and formatting cells and creating the family_friendly DataFrame.

In [6]:
# Columns inspection for data types and check for missing values
print(hazards_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          175 non-null    object
 1   Rattlesnakes  175 non-null    int64 
 2   Ticks         175 non-null    int64 
 3   Posionivy     175 non-null    int64 
 4   Falling       175 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.0+ KB
None


## Cleaning data

**standardizing DB headers**

In [7]:
# renaming all columns in lowercase and replacing all spaces with underscores
trails_data_clean = trails_data.copy()
hazards_data_clean = hazards_data.copy()
trails_data_clean.columns = trails_data_clean.columns.str.lower().str.replace(" ", "_")
hazards_data_clean.columns = hazards_data_clean.columns.str.lower().str.replace(" ", "_")

#renaming also 'name' to 'trail_name' in hazards
hazards_data_clean.rename(columns={'name': 'trail_name'}, inplace=True)
# Display updated headers
print("Updated Trails Data Headers:", list(trails_data_clean.columns))
print("Updated Hazards Data Headers:", list(hazards_data_clean.columns))

Updated Trails Data Headers: ['trail_name', 'trail_type', 'distance', 'high_point', 'elevation_gain', 'difficulty', 'seasons', 'family_friendly', 'backpackable', 'crowded']
Updated Hazards Data Headers: ['trail_name', 'rattlesnakes', 'ticks', 'posionivy', 'falling']


## Cleaning and formating "trail_type"

In [8]:
# standardizing trail types based on the new categories
# Updating 'trail_type' to 'loop' if 'Loop' is in 'trail_name'
trails_data_clean.loc[trails_data_clean['trail_name'].str.contains('Loop', na=False, case=False), 
    'trail_type'
] = 'loop'

# Standardize 'trail_type' values containing "Shuttle" to "Point-to-point"
trails_data_clean.loc[
    trails_data_clean['trail_type'].str.contains('Shuttle', na=False, case=False),
    'trail_type'
] = 'point-to-point'

# Standardize 'trail_type' values containing "Out and Back", "Out-and-back", or "In and out" to "out-and-back"
trails_data_clean.loc[
    trails_data_clean['trail_type'].str.contains('Out and Back|Out-and-back|In and out', na=False, case=False),
    'trail_type'
] = 'out-and-back'

# Standardizing the rest of loop trails. Ensure that 'Loop' and 'Lollipop loop' are consistently lowercased
trails_data_clean.loc[
    trails_data_clean['trail_type'].str.contains('Loop|Lollipop', na=False, case=False), 
    'trail_type'
] = 'loop'

# Display a sample of the cleaned data
trails_data_clean['trail_type'].unique()

array(['loop', 'out-and-back', 'point-to-point', 'ut and Back', nan],
      dtype=object)

In [9]:
# manually changing rest of the excisting categories into desirable format
trails_data_clean.loc[
    trails_data_clean['trail_type'].str.contains('ut and Back', na=False, case=False), 
    'trail_type'
] = 'out-and-back'
trails_data_clean['trail_type'].unique()

array(['loop', 'out-and-back', 'point-to-point', nan], dtype=object)

In [10]:
# Counting the occurrences of each unique trail_type
trail_type_counts = trails_data_clean['trail_type'].value_counts()

# Display the counts
trail_type_counts

trail_type
loop              85
out-and-back      79
point-to-point     7
Name: count, dtype: int64

### Cleaning and formatting "distance" with regex

In [11]:
import re
# Cleaning the 'distance' column by removing all alphabetical characters with regex
trails_data_clean['distance'] = trails_data_clean['distance'].str.replace(r'[a-zA-Z]', '', regex=True).str.strip()

# Converting the 'distance' column to integer
trails_data_clean['distance'] = pd.to_numeric(trails_data_clean['distance'], errors='coerce')

In [12]:
#modifying header "distance" to "distance(miles)"
trails_data_clean.rename(columns={'distance': 'distance(miles)'}, inplace=True)

trails_data_clean

,trail_name,trail_type,distance(miles),high_point,elevation_gain,difficulty,seasons,family_friendly,backpackable,crowded
0,Ainsworth Loop Hike,loop,0.5,150 feet,85 feet,Easy,All year,Yes,No,No
1,Aldrich Butte Hike,out-and-back,13.8,1140 feet,2405 feet,Moderate,All Season,No,No,No
2,Aldrich Butte-Cedar Falls Loop Hike,loop,16.4,"1,140 feet",3105 feet,Difficult,Year round,No,No,No
3,Angels Rest Hike,out-and-back,4.8,1640 feet,1475 feet,Moderate,All Season,Yes,No,Yes
4,Angels Rest-Devils Rest Loop Hike,loop,10.8,2435 feet,3040 feet,Moderate,All Season,Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...
167,Wauna Viewpoint from Tooth Rock Hike,loop,5.4,"1,080 feet",1200 feet,Moderate,Year round,No,No,No
168,Wauneka Point Hike,out-and-back,17.4,"3,872 feet",5680 feet,Difficult,Summer into Fall,No,es,As far as Nesmith Point
169,Wind Mountain Hike,out-and-back,2.9,"1,907 feet",1185 feet,Moderate,Year round,Yes,No,No
170,Wygant Peak Hike,out-and-back,9.2,"2,214 feet",2365 feet,Moderate,all,No,No,Never


### Cleaning and formating "high_point" with regex

In [13]:
# Cleaning the 'high_point' column by removing all alphabetical characters with regex
trails_data_clean['high_point'] = trails_data_clean['high_point'].str.replace(r'[^\d]', '', regex=True).str.strip()

# Convert the 'high_point' column to integer
trails_data_clean['high_point'] = pd.to_numeric(trails_data_clean['high_point'], errors='coerce')

In [14]:
#modifying header "distance" to "distance(miles)"
trails_data_clean.rename(columns={'high_point': 'high_point(feet)'}, inplace=True)

trails_data_clean

,trail_name,trail_type,distance(miles),high_point(feet),elevation_gain,difficulty,seasons,family_friendly,backpackable,crowded
0,Ainsworth Loop Hike,loop,0.5,150.0,85 feet,Easy,All year,Yes,No,No
1,Aldrich Butte Hike,out-and-back,13.8,1140.0,2405 feet,Moderate,All Season,No,No,No
2,Aldrich Butte-Cedar Falls Loop Hike,loop,16.4,1140.0,3105 feet,Difficult,Year round,No,No,No
3,Angels Rest Hike,out-and-back,4.8,1640.0,1475 feet,Moderate,All Season,Yes,No,Yes
4,Angels Rest-Devils Rest Loop Hike,loop,10.8,2435.0,3040 feet,Moderate,All Season,Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...
167,Wauna Viewpoint from Tooth Rock Hike,loop,5.4,1080.0,1200 feet,Moderate,Year round,No,No,No
168,Wauneka Point Hike,out-and-back,17.4,3872.0,5680 feet,Difficult,Summer into Fall,No,es,As far as Nesmith Point
169,Wind Mountain Hike,out-and-back,2.9,1907.0,1185 feet,Moderate,Year round,Yes,No,No
170,Wygant Peak Hike,out-and-back,9.2,2214.0,2365 feet,Moderate,all,No,No,Never


### cleaning and formatting "elevation_gain" with regex

In [15]:
# Cleaning the 'elevation_gain' column by removing all alphabetical characters with regex
trails_data_clean['elevation_gain'] = trails_data_clean['elevation_gain'].str.replace(r'[^\d]', '', regex=True).str.strip()

# Convert the 'elevation_gain' column to integer
trails_data_clean['elevation_gain'] = pd.to_numeric(trails_data_clean['elevation_gain'], errors='coerce')

In [16]:
#modifying header "distance" to "distance(miles)"
trails_data_clean.rename(columns={'elevation_gain': 'elevation_gain(feet)'}, inplace=True)

trails_data_clean

,trail_name,trail_type,distance(miles),high_point(feet),elevation_gain(feet),difficulty,seasons,family_friendly,backpackable,crowded
0,Ainsworth Loop Hike,loop,0.5,150.0,85.0,Easy,All year,Yes,No,No
1,Aldrich Butte Hike,out-and-back,13.8,1140.0,2405.0,Moderate,All Season,No,No,No
2,Aldrich Butte-Cedar Falls Loop Hike,loop,16.4,1140.0,3105.0,Difficult,Year round,No,No,No
3,Angels Rest Hike,out-and-back,4.8,1640.0,1475.0,Moderate,All Season,Yes,No,Yes
4,Angels Rest-Devils Rest Loop Hike,loop,10.8,2435.0,3040.0,Moderate,All Season,Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...
167,Wauna Viewpoint from Tooth Rock Hike,loop,5.4,1080.0,1200.0,Moderate,Year round,No,No,No
168,Wauneka Point Hike,out-and-back,17.4,3872.0,5680.0,Difficult,Summer into Fall,No,es,As far as Nesmith Point
169,Wind Mountain Hike,out-and-back,2.9,1907.0,1185.0,Moderate,Year round,Yes,No,No
170,Wygant Peak Hike,out-and-back,9.2,2214.0,2365.0,Moderate,all,No,No,Never


### Creating family friendly trails DataFrame

In [17]:
#so, now we are ready to create new cleaned data frame with filtered "family_friendly"
#column and download this into our recources folder
columns_to_keep = [
    'trail_name', 'trail_type', 'distance(miles)', 'high_point(feet)',
    'elevation_gain(feet)', 'difficulty', 'seasons', 'family_friendly',
    'backpackable', 'crowded'
]

# Filter the DataFrame for rows where 'family_friendly' contains the word 'yes' (case-insensitive)
family_friendly_df = trails_data_clean[trails_data_clean['family_friendly'].str.contains(r'\byes\b', case=False, na=False)][columns_to_keep].copy()
# Columns to keep in the new DataFrame
columns_to_keep = [
    'trail_name', 'trail_type', 'distance(miles)', 'high_point(feet)',
    'elevation_gain(feet)', 'difficulty', 'seasons', 'family_friendly',
    'backpackable', 'crowded'
]

# Reseting the index of the new DataFrame
family_friendly_df.reset_index(drop=True, inplace=True)


# Display the filtered DataFrame
family_friendly_df

,trail_name,trail_type,distance(miles),high_point(feet),elevation_gain(feet),difficulty,seasons,family_friendly,backpackable,crowded
0,Ainsworth Loop Hike,loop,0.5,150.0,85.0,Easy,All year,Yes,No,No
1,Angels Rest Hike,out-and-back,4.8,1640.0,1475.0,Moderate,All Season,Yes,No,Yes
2,Angels Rest-Devils Rest Loop Hike,loop,10.8,2435.0,3040.0,Moderate,All Season,Yes,No,Yes
3,Balfour-Klickitat Loop Hike,loop,0.7,220.0,115.0,Easy,All year,Yes,No,No
4,Beacon Rock Hike,out-and-back,1.8,848.0,680.0,Moderate,"Year round, except during ice storms",Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...
90,Wahkeena Falls Hike,out-and-back,NaN,645.0,565.0,Moderate,All year,Yes,No,Yes
91,Warren Lake to Mount Defiance Hike,loop,4.7,4959.0,1340.0,Moderate,Summer into fall,"Yes, for older kids",Yes,No
92,Wauna Viewpoint from Eagle Creek Hike,out-and-back,3.4,1080.0,1030.0,Moderate,Year round,Yes,No,No
93,Wind Mountain Hike,out-and-back,2.9,1907.0,1185.0,Moderate,Year round,Yes,No,No


checking and searching for NaN in Data Frame

In [18]:
# Filtering rows where any column contains NaN
nan_rows = family_friendly_df[family_friendly_df.isna().any(axis=1)]

# Display the rows with NaN values
nan_rows

,trail_name,trail_type,distance(miles),high_point(feet),elevation_gain(feet),difficulty,seasons,family_friendly,backpackable,crowded
10,Cape Horn Overlooks Hike,out-and-back,NaN,1300.0,11851795.0,Moderate,All year,Yes,No,Yes
17,Chinidere Mountain Loop Hike,loop,NaN,4673.0,1140.0,Moderate,Early Summer through Fall,Yes,"Yes - connects to Gorge trail network, camping...",Peak summer weekends
23,Eagle Creek to Punch Bowl Falls Hike,out-and-back,NaN,505.0,525.0,Easy,Year round,"Yes, for older kids",No,Yes
40,Klickitat Trail: Lyle to Klickitat Hike,point-to-point,NaN,465.0,340.0,Moderate,All year,Yes,No,No
41,Klickitat Trail: Swale Canyon Hike,point-to-point,NaN,1560.0,1010.0,Moderate,Fall through spring (closed in summer),Yes,No,No
56,Multnomah Falls Hike,NaN,NaN,895.0,870.0,Moderate,Year-round,Yes,No,Very
90,Wahkeena Falls Hike,out-and-back,NaN,645.0,565.0,Moderate,All year,Yes,No,Yes


7 rows: 10, 17, 23, 40, 41, 56, 90 still has some NaNs. 
We will export data and then manually add missing values into exported file .csv.

In [19]:
# Exporting family friendly cleaned DataFrame as a CSV file. 
family_friendly_df.to_csv("resources/cleaned_data/family_friendly_df.csv", index=False)
#exporting hazards_data with formatted headers
hazards_data_clean.to_csv("resources/cleaned_data/hazards_data_clean_df.csv", index=False)

After manually entering all missing data, we’ll import it again and combine it with the hazards DataFrame

In [20]:
# cleaned files to Load 
family_friendly_trails_to_load = Path("resources/cleaned_data/family_friendly_df_v2.csv")

# Read trails and hazards Data Files and store into Pandas DataFrames
family_friendly_trails_final = pd.read_csv(family_friendly_trails_to_load)

family_friendly_trails_final

,trail_name,trail_type,distance(miles),high_point(feet),elevation_gain(feet),difficulty,seasons,family_friendly,backpackable,crowded
0,Ainsworth Loop Hike,loop,0.5,150,85,Easy,All year,Yes,No,No
1,Angels Rest Hike,out-and-back,4.8,1640,1475,Moderate,All Season,Yes,No,Yes
2,Angels Rest-Devils Rest Loop Hike,loop,10.8,2435,3040,Moderate,All Season,Yes,No,Yes
3,Balfour-Klickitat Loop Hike,loop,0.7,220,115,Easy,All year,Yes,No,No
4,Beacon Rock Hike,out-and-back,1.8,848,680,Moderate,"Year round, except during ice storms",Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...
90,Wahkeena Falls Hike,out-and-back,1.4,645,565,Moderate,All year,Yes,No,Yes
91,Warren Lake to Mount Defiance Hike,loop,4.7,4959,1340,Moderate,Summer into fall,"Yes, for older kids",Yes,No
92,Wauna Viewpoint from Eagle Creek Hike,out-and-back,3.4,1080,1030,Moderate,Year round,Yes,No,No
93,Wind Mountain Hike,out-and-back,2.9,1907,1185,Moderate,Year round,Yes,No,No


In [21]:
# checking again if we still have missed rows where any column contains NaN
nan_rows_2 = family_friendly_trails_final[family_friendly_trails_final.isna().any(axis=1)]

# Display the rows with NaN values
if nan_rows_2.empty:
    print("Hooray, we do not have NaN rows!")
else:
    # Display the rows with NaN values
    print("Here are the rows with NaN values:")
    print(nan_rows)

Hooray, we do not have NaN rows!


In [22]:
#checking hazard df
hazards_data_clean

,trail_name,rattlesnakes,ticks,posionivy,falling
0,Ainsworth Loop Hike,0,0,0,0
1,Aldrich Butte Hike,0,1,0,0
2,Aldrich Butte-Cedar Falls Loop Hike,0,1,0,0
3,Angels Rest Hike,0,0,1,0
4,Angels Rest-Devils Rest Loop Hike,0,0,1,0
...,...,...,...,...,...
170,Wauna Viewpoint from Tooth Rock Hike,0,0,1,0
171,Wauneka Point Hike,0,0,0,0
172,Wind Mountain Hike,1,0,1,0
173,Wygant Peak Hike,0,1,1,0


In [23]:
#merging dataframes in one
merged_df = pd.merge(family_friendly_trails_final, hazards_data_clean, on='trail_name', how='left')

# Display the resulting DataFrame
merged_df

,trail_name,trail_type,distance(miles),high_point(feet),elevation_gain(feet),difficulty,seasons,family_friendly,backpackable,crowded,rattlesnakes,ticks,posionivy,falling
0,Ainsworth Loop Hike,loop,0.5,150,85,Easy,All year,Yes,No,No,0,0,0,0
1,Angels Rest Hike,out-and-back,4.8,1640,1475,Moderate,All Season,Yes,No,Yes,0,0,1,0
2,Angels Rest-Devils Rest Loop Hike,loop,10.8,2435,3040,Moderate,All Season,Yes,No,Yes,0,0,1,0
3,Balfour-Klickitat Loop Hike,loop,0.7,220,115,Easy,All year,Yes,No,No,0,1,0,0
4,Beacon Rock Hike,out-and-back,1.8,848,680,Moderate,"Year round, except during ice storms",Yes,No,Yes,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Wahkeena Falls Hike,out-and-back,1.4,645,565,Moderate,All year,Yes,No,Yes,0,0,0,0
91,Warren Lake to Mount Defiance Hike,loop,4.7,4959,1340,Moderate,Summer into fall,"Yes, for older kids",Yes,No,0,0,0,0
92,Wauna Viewpoint from Eagle Creek Hike,out-and-back,3.4,1080,1030,Moderate,Year round,Yes,No,No,0,0,0,0
93,Wind Mountain Hike,out-and-back,2.9,1907,1185,Moderate,Year round,Yes,No,No,1,0,1,0


In [24]:
#exporting merged df
merged_df.to_csv("resources/cleaned_data/merged_df.csv", index=False)


### Creating recommended family trails DataFrame

In [26]:
# filtering rows where all hazard columns have "0" value
recommended_trails_df = merged_df[
    (merged_df['rattlesnakes'] == 0) & 
    (merged_df['ticks'] == 0) & 
    (merged_df['posionivy'] == 0) & 
    (merged_df['falling'] == 0)
]

# Display the resulting DataFrame
print(recommended_trails_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 55 entries, 0 to 94
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   trail_name            55 non-null     object 
 1   trail_type            55 non-null     object 
 2   distance(miles)       55 non-null     float64
 3   high_point(feet)      55 non-null     int64  
 4   elevation_gain(feet)  55 non-null     int64  
 5   difficulty            55 non-null     object 
 6   seasons               55 non-null     object 
 7   family_friendly       55 non-null     object 
 8   backpackable          55 non-null     object 
 9   crowded               55 non-null     object 
 10  rattlesnakes          55 non-null     int64  
 11  ticks                 55 non-null     int64  
 12  posionivy             55 non-null     int64  
 13  falling               55 non-null     int64  
dtypes: float64(1), int64(6), object(7)
memory usage: 6.4+ KB
None


data format is correct for every column

In [27]:
# Exporting recommended for families trails with No hazards as a CSV file. 
recommended_trails_df.to_csv("resources/cleaned_data/recommended_trails_df.csv", index=False)